In [ ]:
!pip install transformers torch pandas numpy scikit-learn plotly tensorflow keras openai textblob nltk emoji spacy gensim


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 41.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install transformers torch gradio nltk textblob scikit-learn pandas numpy plotly

import gradio as gr
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from textblob import TextBlob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from datetime import datetime
import json
import plotly.express as px
import random

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Initialize our models
emotion_classifier = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    return_all_scores=True
)

# Load pre-trained model for response generation
response_tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
response_model = AutoModelForSequenceClassification.from_pretrained("microsoft/DialoGPT-small")

class MentalHealthAssistant:
    def __init__(self):
        self.conversation_history = []
        self.user_emotions = []
        self.load_response_templates()

    def load_response_templates(self):
        """Load pre-defined response templates for different emotional states"""
        self.templates = {
            "sadness": [
                "I hear that you're feeling down. Would you like to talk more about what's troubling you?",
                "It's okay to feel sad. Remember that these feelings are temporary and valid.",
                "I'm here to listen. Would you like to explore some coping strategies together?"
            ],
            "joy": [
                "It's wonderful to hear you're feeling positive! What's bringing you joy right now?",
                "Your happiness is contagious! How can we maintain this positive energy?",
                "I'm glad you're feeling good! Would you like to share more about what's going well?"
            ],
            "anger": [
                "I can sense that you're frustrated. Would you like to talk about what triggered these feelings?",
                "It's natural to feel angry sometimes. Let's explore healthy ways to express these emotions.",
                "I'm here to help you process these feelings. Would you like to try some calming techniques?"
            ],
            "fear": [
                "It sounds like you're feeling anxious. Remember to take deep breaths - we can work through this together.",
                "Your concerns are valid. Would you like to break down what's causing these feelings?",
                "I'm here to support you. Let's explore some ways to manage these anxious thoughts."
            ],
            "neutral": [
                "How are you feeling about things in general?",
                "Would you like to explore any particular thoughts or feelings?",
                "I'm here to listen if you'd like to share more about your experiences."
            ]
        }

    def analyze_emotion(self, text):
        """Analyze the emotional content of text"""
        try:
            emotions = emotion_classifier(text)[0]
            # Get the emotion with highest score
            dominant_emotion = max(emotions, key=lambda x: x['score'])
            return dominant_emotion['label'].lower()
        except Exception as e:
            return "neutral"

    def analyze_sentiment(self, text):
        """Analyze sentiment using TextBlob"""
        analysis = TextBlob(text)
        # Convert polarity to simple labels
        if analysis.sentiment.polarity > 0:
            return "positive"
        elif analysis.sentiment.polarity < 0:
            return "negative"
        return "neutral"

    def get_response(self, text, user_mood):
        """Generate appropriate response based on analysis"""
        # Analyze text
        emotion = self.analyze_emotion(text)
        sentiment = self.analyze_sentiment(text)

        # Store analysis
        self.user_emotions.append({
            'timestamp': datetime.now().isoformat(),
            'emotion': emotion,
            'sentiment': sentiment,
            'mood': user_mood
        })

        # Get appropriate template
        templates = self.templates.get(emotion, self.templates['neutral'])
        response = random.choice(templates)

        # Add resources for negative emotions
        if sentiment == "negative":
            response += "\n\n" + self.get_resources()

        return response, emotion, sentiment

    def get_resources(self):
        """Provide mental health resources"""
        resources = """
        📚 Helpful Resources:

        🆘 Immediate Support:
        - Crisis Line (24/7): 988


        🤝 Free Online Support:
        - 7 Cups: www.7cups.com
        - Mental Health America: www.mhanational.org

        Yoga & Self-Help Tools

        -Guided yoga nidra for deep relaxation
        -Pranayama (yogic breathing) techniques to calm the mind
        -Asana flows for stress relief and grounding
        """
        return resources

    def generate_insights(self):
        """Generate insights from user interaction history"""
        if not self.user_emotions:
            return "Not enough data for analysis."

        # Convert to DataFrame for analysis
        df = pd.DataFrame(self.user_emotions)

        # Emotion distribution
        emotion_counts = df['emotion'].value_counts()

        # Create visualization
        fig = px.pie(
            values=emotion_counts.values,
            names=emotion_counts.index,
            title='Emotional Pattern Distribution'
        )

        return fig

def create_gradio_interface():
    """Create the Gradio interface"""
    assistant = MentalHealthAssistant()

    def respond(message, mood):
        if not message:
            return "Please enter a message.", "", ""

        response, emotion, sentiment = assistant.get_response(message, mood)
        return response, emotion, sentiment

    def show_analysis():
        return assistant.generate_insights()

    # Create interface
    with gr.Blocks(css="footer {visibility: hidden}") as demo:
        gr.Markdown("""
        # 🤗 Mental Health AI Assistant
        *A safe space to talk and get support*

        This AI assistant uses emotion recognition and natural language processing to provide supportive responses
        and track emotional patterns over time.

        **Disclaimer:** This is an educational project and not a replacement for professional mental health care.
        """)

        with gr.Row():
            mood = gr.Dropdown(
                choices=["😊 Happy", "😔 Sad", "😨 Anxious", "😡 Angry", "😐 Neutral"],
                label="How are you feeling?",
                value="😐 Neutral"
            )

        with gr.Row():
            txt = gr.Textbox(
                label="Share your thoughts...",
                lines=3,
                placeholder="Type your message here..."
            )

        with gr.Row():
            submit = gr.Button("Send")
            analyze = gr.Button("Show Analysis")

        with gr.Row():
            output = gr.Textbox(label="Response")
            emotion = gr.Textbox(label="Detected Emotion")
            sentiment = gr.Textbox(label="Sentiment")

        with gr.Row():
            plot = gr.Plot(label="Emotional Patterns")

        # Set up interactions
        submit.click(
            respond,
            inputs=[txt, mood],
            outputs=[output, emotion, sentiment]
        )

        analyze.click(
            show_analysis,
            outputs=plot
        )

    return demo

# Launch the interface
if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at microsoft/DialoGPT-small and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://90bc7f6b435e9c6ba5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
